# VeloVAE benchmark on dyngen data

Notebook benchmarks velocity and latent time inference using VeloVAE on dyngen-generated data.

## Library imports

In [1]:
import velovae as vv

import numpy as np
import pandas as pd
import torch

import anndata as ad
import scvelo as scv

from rgv_tools import DATA_DIR
from rgv_tools.benchmarking import get_velocity_correlation

2024-12-14 02:31:21.846239: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-14 02:31:26.831024: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734139887.884843 3085998 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734139888.345119 3085998 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-14 02:31:29.908273: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## General settings

In [2]:
scv.settings.verbosity = 3

## Constants

In [3]:
torch.manual_seed(0)
np.random.seed(0)

In [4]:
DATASET = "dyngen"

In [5]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / "results").mkdir(parents=True, exist_ok=True)
    (DATA_DIR / DATASET / "processed" / "velovae_vae").mkdir(parents=True, exist_ok=True)

## Velocity pipeline

In [ ]:
velocity_correlation = []

for filename in (DATA_DIR / DATASET / "processed").iterdir():
    if filename.suffix != ".zarr":
        continue

    adata = ad.io.read_zarr(filename)

    try:
        vae = vv.VAE(adata, tmax=20, dim_z=5, device="cuda:0")
        config = {}
        vae.train(adata, config=config, plot=False, embed="pca")

        # Output velocity to adata object
        vae.save_anndata(adata, "vae", DATA_DIR / DATASET / "processed" / "velovae_vae", file_name="velovae.h5ad")

        adata.layers["velocity"] = adata.layers["vae_velocity"].copy()

        velocity_correlation.append(
            get_velocity_correlation(
                ground_truth=adata.layers["true_velocity"], estimated=adata.layers["velocity"], aggregation=np.mean
            )
        )

    except Exception as e:
        # Append np.nan in case of an error and optionally log the error
        print(f"An error occurred: {e}")
        velocity_correlation.append(np.nan)

Estimating ODE parameters...


  0%|          | 0/105 [00:00<?, ?it/s]

Detected 59 velocity genes.
Estimating the variance...


  0%|          | 0/105 [00:00<?, ?it/s]

Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


...

Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/72 [00:00<?, ?it/s]

3 clusters detected based on gene co-expression.
(0.41, 0.08570857886357829), (0.59, 0.9175531644155611)
KS-test result: [2. 2. 0.]
Initial induction: 23, repression: 49/72
Learning Rate based on Data Sparsity: 0.0000
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 2, test iteration: 2
*********       Stage 1: Early Stop Triggered at epoch 5.       *********
*********                      Stage  2                       *********
*********             Velocity Refinement Round 1             *********


  0%|          | 0/300 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.033
Average Set Size: 16
*********     Round 1: Early Stop Triggered at epoch 251.    *********
Change in noise variance: 0.3936
*********             Velocity Refinement Round 2             *********


## Data saving

In [9]:
if SAVE_DATA:
    pd.DataFrame({"velocity": velocity_correlation}).to_parquet(
        path=DATA_DIR / DATASET / "results" / "velovae_vae_correlation.parquet"
    )